<a href="https://colab.research.google.com/github/EddieZ17/colab/blob/main/licenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
/content/drive/MyDrive/cars_dataset

import drive.MyDrive.numeFisier